 TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

# Fine tuning one look

In [1]:
input_width = 160
input_height = 100
channels = 2
class_number = 12
data_path = "D:\\Python\\Wormax_learn2\\preprocessed_data_local_notshuffled\\"
import keras

Q:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models
import keras.backend as K
from keras.applications import Xception

conv_base = Xception(weights='imagenet',
                    include_top=False,
                    input_shape=(input_height, input_width, 3))

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))


def define_model():
    model = models.Sequential(name='test')
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    
    model.add(layers.Dense(class_number, activation='softmax'))
   
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

In [3]:
#conv_base.summary()
conv_base.trainable = False
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 5, 2048)        20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 30720)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               7864576   
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 12)                3084      
Total params: 28,794,932
Trainable params: 7,933,452
Non-trainable params: 20,861,480
_________________________________________________________________


In [4]:
# little prepocessing
from math import atan2, pi

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

In [5]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, num_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        data = np.load(data_dir + listdir[file_i])
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        if shuffle:
            np.random.shuffle(data)
        
        for i in range(0, len(data)-1-batch_size, batch_size):
            samples = data[i:i+batch_size, 0]
            targets = []
            for j in range(batch_size):
                targets.append(
                    to_categorical(get_direction(*data[i+1+j][1][:2]), num_classes=num_classes)
                )
            
            #print(np.array(targets).shape)
            
            # Weird reshape cuz bug
            samples2 = np.zeros((batch_size, input_height, input_width, 3))
            for j, sample in enumerate(samples):
                samples2[j] = sample
            
            # will not work without this
            samples2 = samples2 / 255
            yield samples2, np.array(targets)

In [6]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 1000:
            break
    classes

In [ ]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=64)
validation_generator = generator(data_path, class_number, 'validation', batch_size=64)

print(next(train_generator)[0].shape)

(64, 100, 160, 3)


### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

In [ ]:
model_name = 'worm_transfer_single3'

class_weight = {0: 1.0,
                 1: 1.62,
                 2: 2.68,
                 3: 3.36,
                 4: 2.51,
                 5: 1.53,
                 6: 1.0,
                 7: 1.37,
                 8: 2.16,
                 9: 2.61,
                 10: 2.04,
                 11: 1.3}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir'
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name + '.h5',
        monitor='val_loss',
        save_best_only=True,
    )
]

steps_per_epoch = 200
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks
                        )

Epoch 1/500
200/200 [==============================] - 91s 453ms/step - loss: 4.6171 - actual_acc: 0.0895 - val_loss: 2.4948 - val_actual_acc: 0.0412
Epoch 2/500
200/200 [==============================] - 89s 447ms/step - loss: 4.0773 - actual_acc: 0.1116 - val_loss: 2.4565 - val_actual_acc: 0.1114
Epoch 3/500
200/200 [==============================] - 90s 451ms/step - loss: 4.0214 - actual_acc: 0.1212 - val_loss: 2.4933 - val_actual_acc: 0.0876
Epoch 4/500
200/200 [==============================] - 89s 447ms/step - loss: 4.2639 - actual_acc: 0.0788 - val_loss: 2.5152 - val_actual_acc: 0.0329
Epoch 5/500
200/200 [==============================] - 91s 457ms/step - loss: 4.1645 - actual_acc: 0.0841 - val_loss: 2.4982 - val_actual_acc: 0.1059
Epoch 6/500
200/200 [==============================] - 92s 461ms/step - loss: 4.2197 - actual_acc: 0.0945 - val_loss: 2.4984 - val_actual_acc: 0.0799
Epoch 7/500
200/200 [==============================] - 84s 419ms/step - loss: 4.1488 - actual_acc: 0

200/200 [==============================] - 92s 458ms/step - loss: 4.0108 - actual_acc: 0.1323 - val_loss: 2.4852 - val_actual_acc: 0.0471
Epoch 56/500
200/200 [==============================] - 91s 454ms/step - loss: 4.2501 - actual_acc: 0.0781 - val_loss: 2.5081 - val_actual_acc: 0.0436
Epoch 57/500
200/200 [==============================] - 92s 458ms/step - loss: 4.1419 - actual_acc: 0.0708 - val_loss: 2.4929 - val_actual_acc: 0.0769
Epoch 58/500
200/200 [==============================] - 91s 457ms/step - loss: 4.2208 - actual_acc: 0.1013 - val_loss: 2.5050 - val_actual_acc: 0.0792
Epoch 59/500
200/200 [==============================] - 84s 422ms/step - loss: 4.1388 - actual_acc: 0.0762 - val_loss: 2.4868 - val_actual_acc: 0.0936
Epoch 60/500
200/200 [==============================] - 90s 451ms/step - loss: 4.2441 - actual_acc: 0.0712 - val_loss: 2.4987 - val_actual_acc: 0.1201
Epoch 61/500
200/200 [==============================] - 92s 458ms/step - loss: 4.0835 - actual_acc: 0.1188 